# Cheat Sheet (JijModeling 2 beta)

## Current Status

In [1]:
import jijmodeling as jm
import ommx.v1
import numpy as np
from typing import Tuple

## Sums

### Sum of decision variables

In [2]:
ns = jm.Problem('basic-sum')
problem = jm.Problem("BasicSum", sense=jm.ProblemSense.MINIMIZE)
N = problem.Natural("N")
x = problem.BinaryVar("x", shape=(N,))

# Basic sum
objective = x.sum()
objective

Expression(sum(x))

### Decorator API example

In [3]:
@jm.Problem.define("BasicSum", sense=jm.ProblemSense.MINIMIZE)
def problem(problem: jm.DecoratedProblem):
    N = problem.Placeholder(dtype=jm.DataType.NATURAL)
    x = problem.BinaryVar("x", shape=(N,))
    # Basic sum
    objective = x.sum()
    problem += objective

problem

Problem(name="BasicSum", sense=MINIMIZE, objective=sum(x), constraints=[])

### Weighted sum of decision variables

In [4]:
problem = jm.Problem("WeightedSum", sense=jm.ProblemSense.MINIMIZE)
a = problem.Float("a", ndim=1)
N = problem.DependentVar("N", a.len_at(0))
x = problem.BinaryVar("x", shape=(N,))

# Weighted sum
objective = jm.sum(a * x)
objective

Expression(sum(a * x))

#### Decorator API example

In [5]:
problem = jm.Problem("WeightedSum", sense=jm.ProblemSense.MINIMIZE)

@problem.update
def _(problem: jm.DecoratedProblem):
    a = problem.Placeholder(dtype=jm.DataType.FLOAT, ndim=1)
    N = problem.DependentVar(a.len_at(0))
    x = problem.BinaryVar(shape=(N,))

    # Weighted sum
    objective = (a * x).sum()
    problem += objective

problem

Problem(name="WeightedSum", sense=MINIMIZE, objective=sum(a * x), constraints=[])

### Sum of decision variables along an index set

In [6]:
problem = jm.Problem("SumAlongSet", sense=jm.ProblemSense.MINIMIZE)
N = problem.Natural("N")
x = problem.BinaryVar("x", shape=(N,))
# You can use numpy dtypes as well (bit sizes and endianness are ignored)
C = problem.Natural("C", ndim=1)
# or, equivalently:
# C = problem.Placeholder("C", ndim=1, dtype=np.dtype("u8"))
jm.sum(jm.map(lambda i: x[i], C))

Expression(sum(C.map(lambda i: x[i])))

#### Decorator API example

In [7]:
problem = jm.Problem("SumAlongSet", sense=jm.ProblemSense.MINIMIZE)

@problem.update
def _(problem: jm.DecoratedProblem):
    N = problem.Placeholder(dtype=jm.DataType.NATURAL)
    x = problem.BinaryVar(shape=(N,))
    C = problem.Placeholder(ndim=1, dtype=np.uint64)
    
    problem += jm.sum(x[i] for i in C)

problem

Problem(name="SumAlongSet", sense=MINIMIZE, objective=sum(C.map(lambda (i: natural): x[i])), constraints=[])

### Sum of decision variables over an edge set
#### Tensor whose elements are tuples (recommended)


JijModeling 2 supports placeholders whose elements are tuples, so you can represent a graph edge set as a 1D array of tuples.
This pattern shows up frequently, so a `problem.Graph()` smart constructor is available:

In [8]:
problem = jm.Problem("SumAlongEdgeSet", sense=jm.ProblemSense.MINIMIZE)
V = problem.Natural("V")

E = problem.Graph("E")
# Shorthand for:
# E = problem.Placeholder("E", dtype=Tuple[jm.DataType.NATURAL, jm.DataType.NATURAL], ndim=1)
x = problem.BinaryVar("x", shape=(V,))
jm.map(lambda i, j: x[i] * x[j], E).sum()

Expression(sum(E.map(lambda (i, j): x[i] * x[j])))

By default, `Graph`s are defined to have natural numbers as vertices. You can change vertex type with `vertex` keyword argument: `Graph(vertex=jm.DataType.FLOAT)`.

#### Decorator API example

In [9]:
@jm.Problem.define("SumAlongEdgeSet", sense=jm.ProblemSense.MINIMIZE)
def problem(problem: jm.DecoratedProblem):
    V = problem.Placeholder(dtype=jm.DataType.NATURAL)
    E = problem.Graph()
    x = problem.BinaryVar(shape=(V,))
    
    problem += jm.sum(x[i] * x[j] for (i, j) in E)

problem

Problem(name="SumAlongEdgeSet", sense=MINIMIZE, objective=sum(E.map(lambda ((i, j): Tuple[natural, natural]): x[i] * x[j])), constraints=[])

#### Edge-set variant 1: using rows


Using `rows()` yields a 2D tensor along that axis, allowing a direct sum:

In [10]:
problem = jm.Problem("SumAlongEdgeSet", sense=jm.ProblemSense.MINIMIZE)
N = problem.Natural("N")
V = problem.Natural("V")
E = problem.Natural("E", shape=(N, 2))
x = problem.BinaryVar("x", shape=(V,))
jm.map(lambda i, j: x[i] * x[j], E.rows()).sum()

Expression(sum(E.rows().map(lambda (i, j): x[i] * x[j])))

##### Decorator API example

In [11]:
problem = jm.Problem("SumAlongEdgeSet", sense=jm.ProblemSense.MINIMIZE)

@problem.update
def _(problem: jm.DecoratedProblem):
    V = problem.Placeholder(dtype=np.uint)
    N = problem.Natural()
    # Ideally we'd like to be able to write dtype=V
    E = problem.Placeholder(dtype=jm.DataType.NATURAL, shape=(N, 2))
    x = problem.BinaryVar(shape=(V,))
    problem += jm.sum(x[l] * x[r] for (l, r) in E.rows())

problem

Problem(name="SumAlongEdgeSet", sense=MINIMIZE, objective=sum(E.rows().map(lambda ((l, r): natural[2]): x[l] * x[r])), constraints=[])

#### Edge-set variant 2: naive iteration


Iterate over the length of V.

In [12]:
problem = jm.Problem("SumAlongEdgeSet1", sense=jm.ProblemSense.MINIMIZE)
V = problem.Placeholder("V", dtype=np.uint) # jm.Natural("V") と同値
N = problem.Natural("N")
E = problem.Natural("E", shape=(N, 2))
x = problem.BinaryVar("x", shape=(V,))
jm.sum(jm.map(lambda i: x[E[i, 0]] * x[E[i, 1]], N))

# In the future, you may be able to write something like:
# jm.sum(x[E[:, 0]]  + x[E[:, 1]])

Expression(sum(N.map(lambda i: x[E[i, 0]] * x[E[i, 1]])))

#### Decorator API example

In [13]:
problem = jm.Problem("SumAlongEdgeSet1", sense=jm.ProblemSense.MINIMIZE)

@problem.update
def _(problem: jm.DecoratedProblem):
    V = problem.Placeholder(dtype=np.uint)
    N = problem.Placeholder(dtype=jm.DataType.NATURAL)
    E = problem.Placeholder(dtype=jm.DataType.NATURAL, shape=(N, 2))
    x = problem.BinaryVar(shape=(V,))
    
    problem += jm.sum(x[E[i, 0]] * x[E[i, 1]] for i in N)

problem

Problem(name="SumAlongEdgeSet1", sense=MINIMIZE, objective=sum(N.map(lambda (i: natural): x[E[i, 0]] * x[E[i, 1]])), constraints=[])

### Conditional sum

In [14]:
problem = jm.Problem("ConditionalSum", sense=jm.ProblemSense.MINIMIZE)
N = problem.Natural("N")
J = problem.Float("J", shape=(N, N))
x = problem.BinaryVar("x", shape=(N,))
# This should be
jm.map(
    lambda i: jm.map(lambda j: J[i, j] * x[i] * x[j], i).sum(),
    N
).sum()

Expression(sum(N.map(lambda i: sum(i.map(lambda j: J[i, j] * x[i] * x[j])))))

Alternatively, using `flat_map` you can write:

In [15]:
jm.flat_map(
    lambda i: i.map(lambda j: J[i, j] * x[i] * x[j]),
    N
).sum()

Expression(sum(N.flat_map(lambda i: i.map(lambda j: J[i, j] * x[i] * x[j]))))

#### Decorator API example

In [16]:
problem = jm.Problem("ConditionalSum", sense=jm.ProblemSense.MINIMIZE)

@problem.update
def _(problem: jm.DecoratedProblem):
    N = problem.Placeholder(dtype=jm.DataType.NATURAL)
    J = problem.Placeholder(dtype=jm.DataType.FLOAT, shape=(N, N))
    x = problem.BinaryVar(shape=(N,))

    # Conditional sum with index filtering
    objective = jm.sum(J[i, j] * x[i] * x[j] for i in N for j in i)
    problem += objective

problem

Problem(name="ConditionalSum", sense=MINIMIZE, objective=sum(N.flat_map(lambda (i: natural): i.map(lambda (j: natural): (i, j))).map(lambda ((i, j): Tuple[natural, natural]): J[i, j] * x[i] * x[j])), constraints=[])

### Sum excluding diagonal elements of a matrix

In [17]:
problem = jm.Problem("NonDiagonalSum", sense=jm.ProblemSense.MINIMIZE)
N = problem.Natural("N")
J = problem.Float("J", shape=(N, N))
jm.map(
    lambda i: jm.filter(lambda j: i != j, N).map(lambda j: J[i, j]).sum(),
    N
).sum()

Expression(sum(N.map(lambda i: sum(N.filter(lambda j: i != j).map(lambda j: J[i, j])))))

#### Decorator API example

In [18]:
problem = jm.Problem("NonDiagonalSum", sense=jm.ProblemSense.MINIMIZE)

@problem.update
def _(problem: jm.DecoratedProblem):
    N = problem.Placeholder(dtype=jm.DataType.NATURAL)
    J = problem.Placeholder(dtype=jm.DataType.FLOAT, shape=(N, N))

    objective = jm.sum(J[i, j] for i in N for j in N if i != j)
    problem += objective

problem

Problem(name="NonDiagonalSum", sense=MINIMIZE, objective=sum(N.flat_map(lambda (i: natural): N.map(lambda (j: natural): (i, j))).filter(lambda (i, j): i != j).map(lambda ((i, j): Tuple[natural, natural]): J[i, j])), constraints=[])

Example using `flat_map`:

In [19]:
problem = jm.Problem("NonDiagonalSum", sense=jm.ProblemSense.MINIMIZE)
N = problem.Natural("N")
J = problem.Float("J", shape=(N, N))
jm.flat_map(
    lambda i: jm.filter(lambda j: i != j, N).map(lambda j: J[i, j]),
    N
).sum()

Expression(sum(N.flat_map(lambda i: N.filter(lambda j: i != j).map(lambda j: J[i, j]))))

### Sum depending on another index

In [20]:
problem = jm.Problem("DependentSum", sense=jm.ProblemSense.MINIMIZE)
N = problem.Natural("N")
x = problem.BinaryVar("x", shape=(N,))
a = problem.Natural("a", ndim=1)
# This time, we define M as a dependent variable.
# Any expression can be bound as a dependentVar, and they are printed by
# the variable name. If you print problem, you can also see that the definition of M.
M = problem.DependentVar("M", a.len_at(0))
jm.sum(jm.flat_map(lambda i: a[i].map(lambda j: x[j]), M))

Expression(sum(M.flat_map(lambda i: a[i].map(lambda j: x[j]))))

#### Decorator API example

In [21]:
problem = jm.Problem("DependentSum", sense=jm.ProblemSense.MINIMIZE)

@problem.update
def _(problem: jm.DecoratedProblem):
    N = problem.Placeholder(dtype=jm.DataType.NATURAL)
    x = problem.BinaryVar(shape=(N,))
    a = problem.Placeholder(ndim=1, dtype=jm.DataType.NATURAL)
    M = problem.DependentVar(a.len_at(0))
    
    objective = jm.sum(x[j] for i in M for j in a[i])
    problem += objective

problem

Problem(name="DependentSum", sense=MINIMIZE, objective=sum(M.flat_map(lambda (i: natural): a[i].map(lambda (j: natural): (i, j))).map(lambda ((i, j): Tuple[natural, natural]): x[j])), constraints=[])

## Constraints

### One-hot constraint

In [22]:
problem = jm.Problem("OneHot", sense=jm.ProblemSense.MINIMIZE)
N = problem.Natural("N")
x = problem.BinaryVar("x", shape=(N,))

# One-hot constraint
problem.Constraint("onehot", x.sum() == 1)

Constraint(name="onehot", sense=EQUAL, left=sum(x), right=1, shape=Scalar(Binary))

#### Decorator API example

In [23]:
problem = jm.Problem("OneHot", sense=jm.ProblemSense.MINIMIZE)

@problem.update
def _(problem: jm.DecoratedProblem):
    N = problem.Placeholder(dtype=jm.DataType.NATURAL)
    x = problem.BinaryVar(shape=(N,))

    # One-hot constraint
    problem += problem.Constraint("onehot", x.sum() == 1)

problem

Problem(name="OneHot", sense=MINIMIZE, objective=0, constraints={onehot: [Constraint(name="onehot", sense=EQUAL, left=sum(x), right=1, shape=Scalar(Binary)),],})

### K-hot constraint

In [24]:
problem = jm.Problem("KHot", sense=jm.ProblemSense.MINIMIZE)
N = problem.Natural("N")
K = problem.Natural("K")
x = problem.BinaryVar("x", shape=(N,))

# K-hot constraint
problem.Constraint("k_hot", x.sum() == K)

Constraint(name="k_hot", sense=EQUAL, left=sum(x), right=K, shape=Scalar(Natural))

#### Decorator API example

In [25]:
problem = jm.Problem("KHot", sense=jm.ProblemSense.MINIMIZE)

@problem.update
def _(problem: jm.DecoratedProblem):
    N = problem.Placeholder(dtype=jm.DataType.NATURAL)
    K = problem.Placeholder(dtype=jm.DataType.NATURAL)
    x = problem.BinaryVar(shape=(N,))

    # K-hot constraint
    problem += problem.Constraint("k_hot", x.sum() == K)

problem

Problem(name="KHot", sense=MINIMIZE, objective=0, constraints={k_hot: [Constraint(name="k_hot", sense=EQUAL, left=sum(x), right=K, shape=Scalar(Natural)),],})

### K-hot constraint per column of a 2D binary variable

In [26]:
problem = jm.Problem("2D K-Hot", sense=jm.ProblemSense.MINIMIZE)
K = problem.Natural("K", ndim=1)
# Dependent vars can also be have description and custom latex.
N = problem.DependentVar("N", K.len_at(0), description=r"\# of rows", latex=r"\#K")
M = problem.Natural("M")

x = problem.BinaryVar("x", shape=(N, M))

problem.Constraint("2d k-hot", x.sum(axis=1) == K)

Constraint(name="2d k-hot", sense=EQUAL, left=x.sum(1), right=K, shape=SameTensorLike { spec: Tensor(TensorSpec { shape: [Specified(Located { inner: RangeNat { upper_bound: Located { inner: FVar { name: "N", ext: WithType { type_: Scalar { mode: Static, kind: Natural }, inner: LaTeXMetadata { custom_latex: None, set_style: None, subscript_styles: [] } } }, src_span: Real(RealSrcSpan { filename: "/var/folders/mg/mg6st30d18s7pxjjrk6pkxym0000gn/T/ipykernel_19055/1873889281.py", start: SrcLoc { line: 7, column: 4 }, end: SrcLoc { line: 7, column: 40 } }) } }, src_span: Real(RealSrcSpan { filename: "/var/folders/mg/mg6st30d18s7pxjjrk6pkxym0000gn/T/ipykernel_19055/1873889281.py", start: SrcLoc { line: 7, column: 4 }, end: SrcLoc { line: 7, column: 40 } }) })], element: Located { inner: Scalar { mode: Dynamic, kind: Natural }, src_span: NoSrcSpan } }), scalar: Natural })

#### Decorator API example

In [27]:
problem = jm.Problem("2D K-Hot", sense=jm.ProblemSense.MINIMIZE)

@problem.update
def _(problem: jm.DecoratedProblem):
    K = problem.Placeholder(dtype=jm.DataType.NATURAL, ndim=1)
    N = problem.DependentVar(K.len_at(0), description=r"\# of rows", latex=r"\#K")
    M = problem.Placeholder(dtype=jm.DataType.NATURAL)

    x = problem.BinaryVar(shape=(N, M))

    problem += problem.Constraint("2d k-hot", x.sum(axis=1) == K)

problem

Problem(name="2D K-Hot", sense=MINIMIZE, objective=0, constraints={2d k-hot: [Constraint(name="2d k-hot", sense=EQUAL, left=x.sum(1), right=K, shape=SameTensorLike { spec: Tensor(TensorSpec { shape: [Specified(Located { inner: RangeNat { upper_bound: Located { inner: FVar { name: "N", ext: WithType { type_: Scalar { mode: Static, kind: Natural }, inner: LaTeXMetadata { custom_latex: None, set_style: None, subscript_styles: [] } } }, src_span: Real(RealSrcSpan { filename: "/var/folders/mg/mg6st30d18s7pxjjrk6pkxym0000gn/T/ipykernel_19055/4175410118.py", start: SrcLoc { line: 9, column: 8 }, end: SrcLoc { line: 9, column: 39 } }) } }, src_span: Real(RealSrcSpan { filename: "/var/folders/mg/mg6st30d18s7pxjjrk6pkxym0000gn/T/ipykernel_19055/4175410118.py", start: SrcLoc { line: 9, column: 8 }, end: SrcLoc { line: 9, column: 39 } }) })], element: Located { inner: Scalar { mode: Dynamic, kind: Natural }, src_span: NoSrcSpan } }), scalar: Natural }),],})

### K-hot constraint over each set


In previous versions you specified an Element at the end via `forall=`; in JijModeling 2 you define a set and pass it with `domain=...`.

In [28]:
problem = jm.Problem("KHotOverSet", sense=jm.ProblemSense.MINIMIZE)
N = problem.Natural("N")
C = problem.Natural("C", jagged=True, ndim=2)
M = problem.DependentVar("M", C.len_at(0))
K = problem.Natural("K", shape=(M,))
x = problem.BinaryVar("x", shape=(N,))
problem.Constraint(
    "k-hot_constraint", lambda a: C[a].map(lambda i: x[i]).sum() == K[a], domain=M
)

Constraint(name="k-hot_constraint", , lambda a: sum(C[a].map(lambda (i: natural): x[i])) == K[a], domain=set(M))

#### Decorator API example

In [29]:
problem = jm.Problem("KHotOverSet", sense=jm.ProblemSense.MINIMIZE)

@problem.update
def _(problem: jm.DecoratedProblem):
    N = problem.Placeholder(dtype=jm.DataType.NATURAL)
    C = problem.Placeholder(jagged=True, ndim=2, dtype=jm.DataType.NATURAL)
    M = problem.DependentVar(C.len_at(0))
    K = problem.Placeholder(dtype=jm.DataType.NATURAL, shape=(M,))
    x = problem.BinaryVar(shape=(N,), description="Random binary variable")
    
    constr = problem.Constraint(
        "k-hot_constraint", (jm.sum(x[i] for i in C[a]) == K[a] for a in M),
        description="K-hot constraint over sets; $C_a$ is constrained to have exactly $K_a$ ones.",
    )
    problem += constr

problem

Problem(name="KHotOverSet", sense=MINIMIZE, objective=0, constraints={k-hot_constraint: [Constraint(name="k-hot_constraint", , lambda a: sum(C[a].map(lambda (i: natural): x[i])) == K[a], domain=set(M), description="K-hot constraint over sets; $C_a$ is constrained to have exactly $K_a$ ones."),],})

### Knapsack constraint (linear inequality)

In [30]:
problem = jm.Problem("KnapsackConstraint", sense=jm.ProblemSense.MAXIMIZE)

w = problem.Float("w", ndim=1)
N = problem.DependentVar("N", w.len_at(0))
W = problem.Float("W")
x = problem.BinaryVar("x", shape=(N,))

# Knapsack constraint
constraint = problem.Constraint("weight", (w * x).sum() <= W)
constraint

Constraint(name="weight", sense=LESS_THAN_EQUAL, left=sum(w * x), right=W, shape=Scalar(Float))

In [31]:
problem += constraint

v = problem.Float("v", shape=(N,))
problem += (v * x).sum()
problem

Problem(name="KnapsackConstraint", sense=MAXIMIZE, objective=sum(v * x), constraints={weight: [Constraint(name="weight", sense=LESS_THAN_EQUAL, left=sum(w * x), right=W, shape=Scalar(Float)),],})

In [32]:
v_data = [10, 13, 18, 31, 7, 15]
w_data = [11, 15, 20, 35, 10, 33]
W_data = 47
instance_data = {"v": v_data, "w": w_data, "W": W_data}

# There are two ways to evaluate problem.

# Alternative 1: Directly call eval_* on namespace with instance data.
# Behind the scene, it just creates interpreter, call eval_*.
instance = problem.eval(instance_data)

# Alternative 2: Creating Compiler out of problem and data,
# and call eval_* on interp.
compiler = jm.Compiler.from_problem(problem, instance_data)
instance_2 = compiler.eval_problem(problem)

# The advantage of using a compiler is that 
# you can use helper functions like compiler.get_constraint_id_by_name().

assert instance_2.objective.almost_equal(instance.objective)
assert instance_2.sense == instance.sense
assert len(instance.constraints) == len(instance_2.constraints)

#### Decorator API example

In [33]:
problem = jm.Problem("KnapsackConstraint", sense=jm.ProblemSense.MAXIMIZE)

@problem.update
def _(problem: jm.DecoratedProblem):
    w = problem.Float(ndim=1)
    N = problem.DependentVar(w.len_at(0))
    W = problem.Float()
    x = problem.BinaryVar(shape=(N,))

    # Knapsack constraint
    problem += problem.Constraint("weight", (w * x).sum() <= W)

problem

Problem(name="KnapsackConstraint", sense=MAXIMIZE, objective=0, constraints={weight: [Constraint(name="weight", sense=LESS_THAN_EQUAL, left=sum(w * x), right=W, shape=Scalar(Float)),],})

### SOS1 inequality constraint

In [34]:
problem = jm.Problem("SOS-1", sense=jm.ProblemSense.MINIMIZE)
N = problem.Natural("N")
M = problem.Float("M", shape=(N,), description="Upper bounds for a")

a = problem.ContinuousVar(
    "a",
    shape=N,
    description="Continuous variables subject to the SOS1 constraint",
    lower_bound=0,
    upper_bound=M, # Because M has shape (N,), each element gets its own bound.
    # Equivalent to:
    # upper_bound=lambda i: M[i],
)
x = problem.BinaryVar(
    "x",
    shape=N,
    description="Auxiliary binary variables for the SOS1 constraint on a",
)

problem += problem.Constraint(
    "SOS1",
    x.sum() <= 1,
    description="SOS1 constraint on the auxiliary variables",
)
problem += problem.Constraint(
    "Big-M",
    a <= M * x,
    description="Big-M formulation tying a to the SOS1 auxiliary variables",
)

problem

Problem(name="SOS-1", sense=MINIMIZE, objective=0, constraints={Big-M: [Constraint(name="Big-M", sense=LESS_THAN_EQUAL, left=a, right=M * x, shape=SameTensorLike { spec: Tensor(TensorSpec { shape: [Specified(Located { inner: RangeNat { upper_bound: Located { inner: OverloadedBinaryNumeric { operator: Located { inner: Max, src_span: NoSrcSpan }, lhs: Located { inner: FVar { name: "N", ext: WithType { type_: Scalar { mode: Static, kind: Natural }, inner: LaTeXMetadata { custom_latex: None, set_style: Some(SetStyle { element: None, kind: Natural }), subscript_styles: [] } } }, src_span: Real(RealSrcSpan { filename: "/var/folders/mg/mg6st30d18s7pxjjrk6pkxym0000gn/T/ipykernel_19055/3935419285.py", start: SrcLoc { line: 2, column: 4 }, end: SrcLoc { line: 2, column: 24 } }) }, rhs: Located { inner: OverloadedBinaryNumeric { operator: Located { inner: Max, src_span: NoSrcSpan }, lhs: Located { inner: FVar { name: "N", ext: WithType { type_: Scalar { mode: Static, kind: Natural }, inner: LaTeX

#### Decorator API example

In [35]:
@jm.Problem.define("SOS-1", sense=jm.ProblemSense.MINIMIZE)
def problem(problem: jm.DecoratedProblem):
    N = problem.Natural("N")
    M = problem.Float("M", shape=(N,), description="Upper bounds for a")

    a = problem.ContinuousVar(
        shape=N,
        description="Continuous variables satisfying the SOS1 constraint",
        lower_bound=0,
        upper_bound=M, # Because M has shape (N,), each element gets its own bound.
        # Equivalent to:
        # upper_bound=lambda i: M[i],
    )
    x = problem.BinaryVar(
        shape=N,
        description="Auxiliary binary variables for the SOS1 constraint on a",
    )

    problem += problem.Constraint(
        "SOS1",
        x.sum() <= 1,
        description="SOS1 constraint on the auxiliary variables",
    )
    problem += problem.Constraint(
        "Big-M",
        a <= M * x,
        description="Big-M formulation tying a to the SOS1 auxiliary variables",
    )

problem

Problem(name="SOS-1", sense=MINIMIZE, objective=0, constraints={Big-M: [Constraint(name="Big-M", sense=LESS_THAN_EQUAL, left=a, right=M * x, shape=SameTensorLike { spec: Tensor(TensorSpec { shape: [Specified(Located { inner: RangeNat { upper_bound: Located { inner: OverloadedBinaryNumeric { operator: Located { inner: Max, src_span: NoSrcSpan }, lhs: Located { inner: FVar { name: "N", ext: WithType { type_: Scalar { mode: Static, kind: Natural }, inner: LaTeXMetadata { custom_latex: None, set_style: Some(SetStyle { element: None, kind: Natural }), subscript_styles: [] } } }, src_span: Real(RealSrcSpan { filename: "/var/folders/mg/mg6st30d18s7pxjjrk6pkxym0000gn/T/ipykernel_19055/41884078.py", start: SrcLoc { line: 3, column: 8 }, end: SrcLoc { line: 3, column: 28 } }) }, rhs: Located { inner: OverloadedBinaryNumeric { operator: Located { inner: Max, src_span: NoSrcSpan }, lhs: Located { inner: FVar { name: "N", ext: WithType { type_: Scalar { mode: Static, kind: Natural }, inner: LaTeXMe

### Big-M inequality constraint

In [36]:
problem = jm.Problem("BigM", sense=jm.ProblemSense.MINIMIZE)
N = problem.Natural("N")
c = problem.Float("c", shape=(N, N))

M = problem.Float("M")

x = problem.BinaryVar("x", shape=(N, N))
e = problem.Float("e", shape=(N,))
l = problem.Float("l", shape=(N,))
t = problem.IntegerVar("t", shape=(N,), lower_bound=e, upper_bound=l)
non_diagonals = jm.product(N, N).filter(lambda i, j: i != j)
constraint = problem.Constraint(
    "Big-M",
    lambda i, j: t[i] + c[i, j] - M * (1 - x[i, j]) <= t[j],
    domain=non_diagonals,
)
constraint

Constraint(name="Big-M", , lambda (i, j): t[i] + c[i, j] - M * (1 - x[i, j]) <= t[j], domain=set((N, N)).filter(lambda (i, j): i != j))

In [37]:
problem += constraint
problem

Problem(name="BigM", sense=MINIMIZE, objective=0, constraints={Big-M: [Constraint(name="Big-M", , lambda (i, j): t[i] + c[i, j] - M * (1 - x[i, j]) <= t[j], domain=set((N, N)).filter(lambda (i, j): i != j)),],})

#### Decorator API example

In [38]:
problem = jm.Problem("BigM", sense=jm.ProblemSense.MINIMIZE)

@problem.update
def _(problem: jm.DecoratedProblem):
    N = problem.Placeholder(dtype=jm.DataType.NATURAL)
    c = problem.Float(shape=(N, N))
    M = problem.Float()

    x = problem.BinaryVar(shape=(N, N))
    e = problem.Float(shape=(N,))
    l = problem.Float(shape=(N,))
    t = problem.IntegerVar(shape=(N,), lower_bound=e, upper_bound=l)

    constr = problem.Constraint(
        "Big-M",
        (
            t[i] + c[i, j] - M * (1 - x[i, j]) <= t[j]
            for i in N
            for j in N
            if i != j
        ),
    )
    problem += constr

problem

Problem(name="BigM", sense=MINIMIZE, objective=0, constraints={Big-M: [Constraint(name="Big-M", , lambda (i, j): t[i] + c[i, j] - M * (1 - x[i, j]) <= t[j], domain=N.flat_map(lambda (i: natural): N.map(lambda (j: natural): (i, j))).filter(lambda (i, j): i != j)),],})

### Logistics Planning with Synergy Bonuses
This section illustrates how to formulate problems of the following kind:

1. Objects whose identifiers are not zero-based or contiguous
2. Parameters that are defined only for a subset of combinations of those objects

#### Synergy bonus approach 1: dictionary-based formulation (recommended)

In [39]:
problem = jm.Problem("QuadraticKnapsackLogistics", sense=jm.ProblemSense.MAXIMIZE)

@problem.update
def _(problem: jm.DecoratedProblem):
    # --- 3. Parameters ---
    # I = problem.Natural("I", ndim=1, description="The set of parcels")
    # J = problem.Natural("J", ndim=1, description="The set of trucks")
    # A Category represents the runtime-supplied set of labels (integers or strings).
    I = problem.CategoryLabel("I", description="The labels of parcels")
    J = problem.CategoryLabel(description="The labels of trucks")

    # To index by a Category you need to declare the data type as a dictionary.
    # Its internal representation therefore differs from tensors and is stored as a dict.

    # Using `dict_keys` instead of `shape`/`ndim` declares a dictionary keyed by the supplied types.
    # By default the compiler expects values to be defined for every key in that domain.
    weights = problem.Integer(
        "w", dict_keys=I, description="The weight of each parcel"
    )
    base_revenues = problem.Integer(
        "r", dict_keys=I, description="The base revenue of each parcel"
    )
    capacities = problem.Integer(
        "C", dict_keys=J, description="The capacity of each truck"
    )

    # Setting `partial_dict=True` allows the dictionary to be defined only on a subset of keys.
    # Here `s` is defined only on the parcel pairs that actually have a synergy bonus.
    synergy_bonuses = problem.Integer(
        "s",
        dict_keys=(I, I),
        partial_dict=True,
        description="The synergy bonus between pairs of parcels",
    )

    # Alternatively, using the syntactic sugar:
    # synergy_bonus = problem.PartialDict(
    #     "s",
    #     dtype=int,
    #     keys=(I, I),
    #     description="The synergy bonus between pairs of parcels",
    # )

    # --- 4. Decision Variables ---
    # The number of decision variables has to be determined statically from placeholders,
    # so dictionaries of decision variables must be defined on the entire key domain (total).
    x = problem.BinaryVar(
        dict_keys=(I, J),
        description="x[i,j] = 1 if parcel i is assigned to truck j, else 0",
    )
    # If you only assign parcels that participate in a synergy bonus (unlikely, but possible),
    # you could instead write:
    # x = problem.BinaryVar(
    #     "x",
    #     dict_keys=s.keys(),
    #     description="x[i,j] = 1 if parcel i is assigned to truck j, else 0",
    # )

    # --- 5. Objective Function ---
    problem += jm.sum(
        synergy_bonuses[i, k] * x[i, j] * x[k, j]
        for j in J
        # Use keys() to iterate over keys,
        # items() key-value-pairs, and
        # values() for values.
        for (i, k) in synergy_bonuses.keys()
    ) + jm.sum(base_revenues[i] * x[i, j] for i in I for j in J)

    # --- 6. Constraints ---
    problem += problem.Constraint(
        "parcel_assign", [jm.sum(x[i, j] for j in J) == 1 for i in I]
    )
    problem += problem.Constraint(
        "truck_capacity",
        [jm.sum(weights[i] * x[i, j] for i in I) <= capacities[j] for j in J],
    )

problem

Problem(name="QuadraticKnapsackLogistics", sense=MAXIMIZE, objective=sum(J.flat_map(lambda (j: CategoryLabel("J")): s.keys().map(lambda ((i, k): Tuple[CategoryLabel("I"), CategoryLabel("I")]): (j, i, k))).map(lambda ((j, i, k): Tuple[CategoryLabel("J"), CategoryLabel("I"), CategoryLabel("I")]): s[i, k] * x[i, j] * x[k, j])) + sum(I.flat_map(lambda (i: CategoryLabel("I")): J.map(lambda (j: CategoryLabel("J")): (i, j))).map(lambda ((i, j): Tuple[CategoryLabel("I"), CategoryLabel("J")]): r[i] * x[i, j])), constraints={parcel_assign: [Constraint(name="parcel_assign", , lambda i: sum(J.map(lambda (j: CategoryLabel("J")): x[i, j])) == 1, domain=I),],truck_capacity: [Constraint(name="truck_capacity", , lambda j: sum(I.map(lambda (i: CategoryLabel("I")): w[i] * x[i, j])) <= C[j], domain=J),],})

In [40]:
synergies_data = {
    (1, 3): 25,
    (2, 5): 30,
    (2, 6): 20,
    (4, 8): 40,
    (5, 7): 22,
}
percels_data = [1, 2, 3, 4, 5, 6, 7, 8]
trucks_data = ["Truck A", "Truck B", "Truck C"]
r_data = {1: 50, 2: 75, 3: 40, 4: 80, 5: 60, 6: 65, 7: 35, 8: 90}
weight_data = {1: 35, 2: 45, 3: 25, 4: 50, 5: 30, 6: 40, 7: 20, 8: 55}
capacity_data = {"Truck A": 100, "Truck B": 120, "Truck C": 80}
data = {
    "I": percels_data,
    "J": trucks_data,
    "w": weight_data,
    "r": r_data,
    "C": capacity_data,
    "s": synergies_data,
}
compiler = jm.Compiler.from_problem(problem, data)
instance = compiler.eval_problem(problem)

x_var = {
    (i, j): compiler.get_decision_variable_by_name("x", (i, j))
    for i in percels_data
    for j in trucks_data
}

expected_objective = ommx.v1.Function(
    sum(
        synergies_data[i, k] * x_var[i, j] * x_var[k, j]
        for j in trucks_data
        for (i, k) in synergies_data
    )
    + sum(r_data[i] * x_var[i, j] for i in percels_data for j in trucks_data)
)

assert instance.objective.almost_equal(expected_objective)

percel_constrs = compiler.get_constraint_id_by_name("parcel_assign")
for i in percels_data:
    c_id = percel_constrs[(i,)]
    assert c_id is not None
    constr = instance.constraints[c_id]
    assert constr is not None
    assert constr.equality == ommx.v1.Constraint.EQUAL_TO_ZERO
    expected_function = ommx.v1.Function(sum(x_var[i, j] for j in trucks_data) - 1)
    assert constr.function.almost_equal(expected_function)

capacity_constrs = compiler.get_constraint_id_by_name("truck_capacity")
for j in trucks_data:
    c_id = capacity_constrs[(j,)]
    assert c_id is not None
    constr = instance.constraints[c_id]
    assert constr is not None
    assert constr.equality == ommx.v1.Constraint.LESS_THAN_OR_EQUAL_TO_ZERO
    expected_function = ommx.v1.Function(
        sum(weight_data[i] * x_var[i, j] for i in percels_data) - capacity_data[j]
    )
    assert constr.function.almost_equal(expected_function)

#### Approach 2: tensor definition via array min/max

In [41]:
problem = jm.Problem("QuadraticKnapsackLogistics", sense=jm.ProblemSense.MAXIMIZE)

@problem.update
def _(problem: jm.DecoratedProblem):
    # --- 3. Parameters ---
    I = problem.Natural("I", ndim=1, description="The set of parcels")
    J = problem.Natural("J", ndim=1, description="The set of trucks")
    P = problem.Placeholder("P", ndim=1, dtype=Tuple[np.uint, np.uint], description="The set of parcel pairs with a synergy bonus")
    weights = problem.Integer(
        "w", ndim=1, description="The weight of each parcel"
    )
    base_revenues = problem.Integer(
        "r", ndim=1, description="The base revenue of each parcel"
    )
    capacities = problem.Integer(
        "C", ndim=1, description="The capacity of each truck"
    )
    synergy_bonuses = problem.Integer(
        "s",
        ndim=2,
        description="The synergy bonus between pairs of parcels",
    )

    # --- 4. Decision Variables ---
    x = problem.BinaryVar(
        "x",
        shape=(I.max() + 1, J.max() + 1),
        description="x[i,j] = 1 if parcel i is assigned to truck j, else 0",
    )

    # --- 5. Objective Function ---
    problem += jm.sum(
        synergy_bonuses[i, k] * x[i, j] * x[k, j]
        for j in J
        for (i, k) in P
    ) + jm.sum(
        base_revenues[i] * x[i, j]
        for i in I
        for j in J
    )

    # --- 6. Constraints ---
    problem += problem.Constraint(
        "parcel_assign", [jm.sum(x[i, j] for j in J) == 1 for i in I]
    )
    problem += problem.Constraint(
        "truck_capacity",
        [
            jm.sum(weights[i] * x[i, j] for i in I) <= capacities[j]
            for j in J
        ],
    )

problem

Problem(name="QuadraticKnapsackLogistics", sense=MAXIMIZE, objective=sum(J.flat_map(lambda (j: natural): P.map(lambda ((i, k): Tuple[natural, natural]): (j, i, k))).map(lambda ((j, i, k): Tuple[natural, natural, natural]): s[i, k] * x[i, j] * x[k, j])) + sum(I.flat_map(lambda (i: natural): J.map(lambda (j: natural): (i, j))).map(lambda ((i, j): Tuple[natural, natural]): r[i] * x[i, j])), constraints={parcel_assign: [Constraint(name="parcel_assign", , lambda i: sum(J.map(lambda (j: natural): x[i, j])) == 1, domain=set(I)),],truck_capacity: [Constraint(name="truck_capacity", , lambda j: sum(I.map(lambda (i: natural): w[i] * x[i, j])) <= C[j], domain=set(J)),],})